# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 29 2022 8:30AM,243022,10,CTF0011340,"Citieffe, Inc.",Released
1,Jul 28 2022 3:30PM,243021,16,SHL-8335476,"SHL Pharma, LLC",Released
2,Jul 28 2022 3:14PM,243018,10,0085914581,ISDIN Corporation,Released
3,Jul 28 2022 3:14PM,243018,10,0085914583,ISDIN Corporation,Released
4,Jul 28 2022 3:14PM,243018,10,0085914582,ISDIN Corporation,Released
5,Jul 28 2022 3:14PM,243018,10,0085914584,ISDIN Corporation,Released
6,Jul 28 2022 3:14PM,243018,10,0085914643,ISDIN Corporation,Released
7,Jul 28 2022 3:14PM,243018,10,0085914653,ISDIN Corporation,Released
8,Jul 28 2022 3:14PM,243018,10,0085914654,ISDIN Corporation,Released
9,Jul 28 2022 3:14PM,243018,10,0085914658,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,243015,Released,1
24,243017,Released,1
25,243018,Released,8
26,243021,Released,1
27,243022,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243015,NaN,NaN,1.0
243017,NaN,NaN,1.0
243018,NaN,NaN,8.0
243021,NaN,NaN,1.0
243022,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242901,1.0,0.0,5.0
242907,0.0,0.0,1.0
242908,2.0,1.0,0.0
242914,0.0,0.0,1.0
242921,35.0,13.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242901,1,0,5
242907,0,0,1
242908,2,1,0
242914,0,0,1
242921,35,13,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242901,1,0,5
1,242907,0,0,1
2,242908,2,1,0
3,242914,0,0,1
4,242921,35,13,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242901,1,,5
1,242907,,,1
2,242908,2,1,
3,242914,,,1
4,242921,35,13,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 29 2022 8:30AM,243022,10,"Citieffe, Inc."
1,Jul 28 2022 3:30PM,243021,16,"SHL Pharma, LLC"
2,Jul 28 2022 3:14PM,243018,10,ISDIN Corporation
10,Jul 28 2022 3:12PM,243017,22,"NBTY Global, Inc."
11,Jul 28 2022 3:10PM,243015,22,"NBTY Global, Inc."
12,Jul 28 2022 2:58PM,243011,22,"NBTY Global, Inc."
13,Jul 28 2022 2:40PM,243008,21,"NBTY Global, Inc."
14,Jul 28 2022 2:30PM,243006,21,"NBTY Global, Inc."
15,Jul 28 2022 2:28PM,243005,21,"NBTY Global, Inc."
16,Jul 28 2022 1:47PM,243003,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 29 2022 8:30AM,243022,10,"Citieffe, Inc.",,,1
1,Jul 28 2022 3:30PM,243021,16,"SHL Pharma, LLC",,,1
2,Jul 28 2022 3:14PM,243018,10,ISDIN Corporation,,,8
3,Jul 28 2022 3:12PM,243017,22,"NBTY Global, Inc.",,,1
4,Jul 28 2022 3:10PM,243015,22,"NBTY Global, Inc.",,,1
5,Jul 28 2022 2:58PM,243011,22,"NBTY Global, Inc.",,,1
6,Jul 28 2022 2:40PM,243008,21,"NBTY Global, Inc.",,,1
7,Jul 28 2022 2:30PM,243006,21,"NBTY Global, Inc.",,,1
8,Jul 28 2022 2:28PM,243005,21,"NBTY Global, Inc.",,,1
9,Jul 28 2022 1:47PM,243003,16,Sartorius Bioprocess Solutions,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 29 2022 8:30AM,243022,10,"Citieffe, Inc.",1,,
1,Jul 28 2022 3:30PM,243021,16,"SHL Pharma, LLC",1,,
2,Jul 28 2022 3:14PM,243018,10,ISDIN Corporation,8,,
3,Jul 28 2022 3:12PM,243017,22,"NBTY Global, Inc.",1,,
4,Jul 28 2022 3:10PM,243015,22,"NBTY Global, Inc.",1,,
5,Jul 28 2022 2:58PM,243011,22,"NBTY Global, Inc.",1,,
6,Jul 28 2022 2:40PM,243008,21,"NBTY Global, Inc.",1,,
7,Jul 28 2022 2:30PM,243006,21,"NBTY Global, Inc.",1,,
8,Jul 28 2022 2:28PM,243005,21,"NBTY Global, Inc.",1,,
9,Jul 28 2022 1:47PM,243003,16,Sartorius Bioprocess Solutions,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 29 2022 8:30AM,243022,10,"Citieffe, Inc.",1,,
1,Jul 28 2022 3:30PM,243021,16,"SHL Pharma, LLC",1,,
2,Jul 28 2022 3:14PM,243018,10,ISDIN Corporation,8,,
3,Jul 28 2022 3:12PM,243017,22,"NBTY Global, Inc.",1,,
4,Jul 28 2022 3:10PM,243015,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 29 2022 8:30AM,243022,10,"Citieffe, Inc.",1.0,NaN,NaN
1,Jul 28 2022 3:30PM,243021,16,"SHL Pharma, LLC",1.0,NaN,NaN
2,Jul 28 2022 3:14PM,243018,10,ISDIN Corporation,8.0,NaN,NaN
3,Jul 28 2022 3:12PM,243017,22,"NBTY Global, Inc.",1.0,NaN,NaN
4,Jul 28 2022 3:10PM,243015,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 29 2022 8:30AM,243022,10,"Citieffe, Inc.",1.0,0.0,0.0
1,Jul 28 2022 3:30PM,243021,16,"SHL Pharma, LLC",1.0,0.0,0.0
2,Jul 28 2022 3:14PM,243018,10,ISDIN Corporation,8.0,0.0,0.0
3,Jul 28 2022 3:12PM,243017,22,"NBTY Global, Inc.",1.0,0.0,0.0
4,Jul 28 2022 3:10PM,243015,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1214893,20.0,2.0,0.0
16,1214863,5.0,0.0,0.0
19,485829,9.0,14.0,37.0
20,485865,21.0,9.0,10.0
21,729019,3.0,0.0,0.0
22,972019,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1214893,20.0,2.0,0.0
1,16,1214863,5.0,0.0,0.0
2,19,485829,9.0,14.0,37.0
3,20,485865,21.0,9.0,10.0
4,21,729019,3.0,0.0,0.0
5,22,972019,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,20.0,2.0,0.0
1,16,5.0,0.0,0.0
2,19,9.0,14.0,37.0
3,20,21.0,9.0,10.0
4,21,3.0,0.0,0.0
5,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,20.0
1,16,Released,5.0
2,19,Released,9.0
3,20,Released,21.0
4,21,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21,22
Status,,,,,,
Completed,0.0,0.0,37.0,10.0,0.0,0.0
Executing,2.0,0.0,14.0,9.0,0.0,0.0
Released,20.0,5.0,9.0,21.0,3.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21,22
0,Completed,0.0,0.0,37.0,10.0,0.0,0.0
1,Executing,2.0,0.0,14.0,9.0,0.0,0.0
2,Released,20.0,5.0,9.0,21.0,3.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21,22
0,Completed,0.0,0.0,37.0,10.0,0.0,0.0
1,Executing,2.0,0.0,14.0,9.0,0.0,0.0
2,Released,20.0,5.0,9.0,21.0,3.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()